In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
import os
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils,datasets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from torchvision.io import read_image


In [2]:
# import zipfile
# with zipfile.ZipFile('/content/drive/MyDrive/Machine Learning/archive (1).zip', 'r') as zip_ref:
#     zip_ref.extractall()
import gc

gc.collect()

torch.cuda.empty_cache()

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Resize(224)])
trainset = datasets.ImageFolder(root='../input/fruit-dataset/fruits-360_dataset/fruits-360/Training', transform=transform)
testset = datasets.ImageFolder(root='../input/fruit-dataset/fruits-360_dataset/fruits-360/Test', transform=transform)

In [4]:
# train_dataset, test_dataset = torch.utils.data.random_split(totalset, (train_count, test_count))

In [5]:
batch_SIZE = 64
train_dataset_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_SIZE, shuffle=True, num_workers=1)
# valid_dataset_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_SIZE, shuffle=True, num_workers=1) 
test_dataset_loader  = torch.utils.data.DataLoader(testset , batch_size=batch_SIZE, shuffle=False,num_workers=1)
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# dataloaders = {'train': train_dataset_loader, 'test': test_dataset_loader}

In [6]:
# dataiter = iter(train_dataset_loader)

# images, labels = dataiter.next()

# # show images


In [7]:

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 1
        self.conv1a = nn.Sequential(
            nn.Conv2d(3,64,3,padding=1),nn.BatchNorm2d(64),nn.ReLU()
        )
        self.conv1b = nn.Sequential(
            nn.Conv2d(64,64,3,padding=1),nn.MaxPool2d(2,2),nn.BatchNorm2d(64),nn.ReLU()
        )
        # 2
        self.conv2a = nn.Sequential(
            nn.Conv2d(64,128,3,padding=1),nn.BatchNorm2d(128),nn.ReLU()
        )
        self.conv2b = nn.Sequential(
            nn.Conv2d(128,128,3,padding=1),nn.MaxPool2d(2,2),nn.BatchNorm2d(128),nn.ReLU()
        )
        # 3
        self.conv3a = nn.Sequential(
            nn.Conv2d(128,256,3,padding=1),nn.BatchNorm2d(256),nn.ReLU()
        )
        self.conv3b = nn.Sequential(
                nn.Conv2d(256,256,3,padding=1),nn.MaxPool2d(2,2),nn.BatchNorm2d(256),nn.ReLU()
        )
        # 4
        self.conv4a = nn.Sequential(
            nn.Conv2d(256,512,3,padding=1),nn.BatchNorm2d(512),nn.ReLU()
        )
        self.conv4b = nn.Sequential(
            nn.Conv2d(512,512,3,padding=1),nn.BatchNorm2d(512),nn.ReLU()
        )
        self.conv4c = nn.Sequential(
            nn.Conv2d(512,512,3,padding=1),nn.MaxPool2d(2,2),nn.BatchNorm2d(512),nn.ReLU()
        )
        # 5
        self.conv5a = nn.Sequential(
            nn.Conv2d(512,512,3,padding=1),nn.BatchNorm2d(512),nn.ReLU()
        )
        self.conv5b = nn.Sequential(
            nn.Conv2d(512,512,3,padding=1),nn.BatchNorm2d(512),nn.ReLU()
        )
        self.conv5c = nn.Sequential(
            nn.Conv2d(512,512,3,padding=1),nn.MaxPool2d(2,2),nn.BatchNorm2d(512),nn.ReLU()
        )
        self.fc1 =  nn.Sequential(
            nn.Linear(25088,4096),nn.ReLU()
        )
        self.fc2 =  nn.Sequential(
            nn.Linear(4096,4096),nn.ReLU()
        )
        self.fc3 =  nn.Sequential(
            nn.Linear(4096,131),nn.ReLU()
        )
        
    def forward(self, x):
        #print(x.shape)
        out = self.conv1a(x)
        out = self.conv1b(out)
        #print(out.shape)
        out = self.conv2a(out)
        out = self.conv2b(out)
        #print(out.shape)
        out = self.conv3a(out)
        out = self.conv3b(out)
        #print(out.shape)
        out = self.conv4a(out)
        out = self.conv4b(out)
        out = self.conv4c(out)
        out = self.conv5a(out)
        out = self.conv5b(out)
        out = self.conv5c(out)
        out = torch.flatten(out, 1)
          # print(out.shape)
          # out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out
net = Net()

In [8]:
# print(type(trainset))
# print(len(totalset))

In [9]:
# print(train_count)
# print(test_count)
# # print(valid_count)

In [10]:
# net.state_dict()
net = net.to(DEVICE)

In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [12]:
print(optimizer.state_dict())

{'state': {}, 'param_groups': [{'lr': 0.001, 'momentum': 0.9, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]}]}


In [13]:
startepoch=0
if os.path.exists('./checkpoint.pth'):
  print("Loading from existing model weights..")
  state= torch.load("./checkpoint.pth")
  if state :
    startepoch=state["epoch"]
    net.load_state_dict(state['model_state_dict'])
    optimizer.load_state_dict(state['optimizer_state_dict'])
else:
    print("Runnig first time")
num_epochs = 15
for epoch in range(startepoch,num_epochs):
  running_loss = 0.0
  for i, data in enumerate(train_dataset_loader):
    inputs, labels = data
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)
    optimizer.zero_grad()
    outputs = net(inputs)
#     print(outputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
            .format(epoch+1, num_epochs, i+1, num_epochs, loss.item())) # 2 para num_epoch= total_step
  print('Epoch -> ',epoch)
  torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, "./checkpoint.pth")

print('Finished Training')

    

Runnig first time
Epoch [1/15], Step [100/15], Loss: 4.1957
Epoch [1/15], Step [200/15], Loss: 3.1792
Epoch [1/15], Step [300/15], Loss: 1.9105
Epoch [1/15], Step [400/15], Loss: 1.7224
Epoch [1/15], Step [500/15], Loss: 0.9606
Epoch [1/15], Step [600/15], Loss: 1.2916
Epoch [1/15], Step [700/15], Loss: 1.0073
Epoch [1/15], Step [800/15], Loss: 1.0134
Epoch [1/15], Step [900/15], Loss: 0.8167
Epoch [1/15], Step [1000/15], Loss: 1.1627
Epoch ->  0
Epoch [2/15], Step [100/15], Loss: 0.7787
Epoch [2/15], Step [200/15], Loss: 1.2376
Epoch [2/15], Step [300/15], Loss: 0.7063
Epoch [2/15], Step [400/15], Loss: 0.7797
Epoch [2/15], Step [500/15], Loss: 0.8520
Epoch [2/15], Step [600/15], Loss: 1.3778
Epoch [2/15], Step [700/15], Loss: 0.9457
Epoch [2/15], Step [800/15], Loss: 1.1512
Epoch [2/15], Step [900/15], Loss: 0.7687
Epoch [2/15], Step [1000/15], Loss: 1.0726
Epoch ->  1
Epoch [3/15], Step [100/15], Loss: 0.6943
Epoch [3/15], Step [200/15], Loss: 1.0705
Epoch [3/15], Step [300/15], Los

In [14]:
print(DEVICE)

cuda:0


In [15]:
net.eval()

Net(
  (conv1a): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv1b): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (conv2a): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (conv2b): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Re

In [16]:
# testing
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataset_loader:
        images, labels = data
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * (correct / total)))

Accuracy of the network on the test images: 79 %
